In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 109.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 28.2 MB/s eta 0:00:

In [2]:
import pandas as pd
from simpletransformers.t5 import T5Model

In [3]:
## Read data
train_df = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_train.csv").astype(str)[['prefix','input_text','target_text']]
eval_df = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_val.csv").astype(str)[['prefix','input_text','target_text']]
test_df = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_test.csv").astype(str)[['prefix','input_text','target_text']]

## Select data-points with prefix = 'explanation'
train_df = train_df[train_df['prefix']=='explanation'].copy()
eval_df = eval_df[eval_df['prefix']=='explanation'].copy()
test_df = test_df[test_df['prefix']=='explanation'].copy()

train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

print("Train shape: ", train_df.shape)
print("Val shape: ", eval_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (14057, 3)
Val shape:  (1786, 3)
Test shape:  (1759, 3)


In [14]:
model_args = {
    "max_seq_length": 250,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "num_train_epochs": 5,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "fp16": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "learning_rate":1e-4,
    "weight_decay":0.01
}

In [13]:
eval_df

,prefix,input_text,target_text
0,explanation,no to broke girlfriends the ghetto,"the phrase ""no to broke girlfriends"" is derog..."
1,explanation,bingo hates americans hates our values hates o...,bingo expressing hatred towards americans usi...
2,explanation,don lemon is a nigger,this sentence uses a derogatory term towards ...
3,explanation,hard to believe discord is hopping on the impo...,this sentence includes hateful language towar...
4,explanation,luv seeing white ppl telling indigenous folks ...,this sentence perpetuates racism against indi...
...,...,...,...
1781,explanation,does that fucking niggerfaggot of barry soetor...,this sentence contains derogatory language to...
1782,explanation,come together as humans means join the african...,this sentence presents a clear case of anti-m...
1783,explanation,guilty as fuckin charged fuck dat white bitch,the use of profanity and disrespectful langua...
1784,explanation,have at it i am white and glad of it niggers h...,this sentence contains several derogatory ter...


In [15]:
model = T5Model("t5", "t5-base", args=model_args)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [16]:
model.train_model(train_df, eval_data=eval_df, output_dir="/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation")

  0%|          | 0/14057 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/879 [00:00<?, ?it/s]

  0%|          | 0/1786 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Running Epoch 4 of 5:   0%|          | 0/879 [00:00<?, ?it/s]

  0%|          | 0/1786 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

(4395,
 {'global_step': [879, 1758, 2637, 3516, 4395],
  'eval_loss': [1.8763089637671198,
   1.768275260925293,
   1.716676162821906,
   1.6871539428830147,
   1.6740181595087051],
  'train_loss': [1.8860676288604736,
   1.7961878776550293,
   1.6072434186935425,
   1.7774335145950317,
   1.5114473104476929]})

In [17]:
model_args2 = {
    "overwrite_output_dir": True,
    "max_seq_length": 250,
    "eval_batch_size": 32,
    "use_multiprocessing": False,
    "num_beams": 3,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

# Load the trained model
model = T5Model("t5", "/content/outputs/best_model", args=model_args2)


In [18]:
# Prepare the data for testing
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(test_df["prefix"].tolist(), test_df["input_text"].tolist())
]
truth = test_df["target_text"].tolist()
tasks = test_df["prefix"].tolist()

In [19]:
# Get the model predictions
preds = model.predict(to_predict)

Generating outputs:   0%|          | 0/55 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3745: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/5277 [00:00<?, ?it/s]

In [20]:
## Taking only the top (first) predictions
top_preds = [pred[0] for pred in preds]
test_df["predicted"] = top_preds
test_df['all_preds'] = preds

## Saving predictions
test_df.to_csv('/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation/df_test_predictions_explanation_baseline.csv', index=False)


In [24]:
test_df.iloc[0]['target_text']

' this sentence uses derogatory terms to refer to two different groups. "heeb" is an ethnic slur used against jews, implying their supposed usury or greediness. "mura masa" may be referring to yusuf ibn tashfin, leader of the almoravid empire, who was known as murabitun, but the term used seems incorrect or misspelled. the usage suggests a lack of knowledge or interest in diverse histories beyond one\'s own prejudices or biases towards these communities. it implies hostility based on race and faith.'

In [25]:
test_df.iloc[0]['predicted']

'this sentence contains derogatory language towards two different groups of people. the term "heeb" is a slur used against jews, while "muramasa" is a sexist s'

In [26]:
!mv '/content/outputs' '/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation'